## Instituto Federal do Sudeste de Minas Gerais, Campus Barbacena
### Projeto Laboratório de Redes de Conhecimento

## Curso de Mineração de Dados Aplicada

Prof. Rafael José de Alencar Almeida
<rafael.alencar@ifsudestemg.edu.br>

<br>

## Aula 7:  Agrupamentos

<img src="https://i.imgur.com/S65Sk9c.jpg">
**Fonte da figura:** https://mubaris.com/posts/kmeans-clustering/

In [3]:
import pandas as pd

# Carrega o dataset de notícias do IF

df = pd.read_csv('./data/df_noticias_if.csv')
df['doc'] = df.apply(lambda row: row['titulo'] + ' ' + row['conteudo'], axis=1)
df['doc'].head()

FileNotFoundError: File b'./data/df_noticias_if.csv' does not exist

In [15]:
import re
import nltk
from string import punctuation

# Pré-processamento

numbers = '0123456789'
stopwords = nltk.corpus.stopwords.words('portuguese')

def preprocessa(txt):
    txt = txt.lower()
    txt = ''.join([c for c in txt if c not in punctuation + numbers])
    txt = re.sub(r'\n|\r|\t', '', txt)
    txt = re.sub(r' .+? ', ' ', txt)
    txt = ' '.join([t for t in txt.split(' ') if t not in stopwords])
    
    return txt

df['doc'] = df['doc'].apply(preprocessa)
df['doc'].head()

0    campus divulga provisório viii barbacena provi...
1    divulgado edital convocação assembleia centro ...
2    pesquisador bélgica palestra campus luc fará p...
3    divulgada homologação inscrições candidatura c...
4    aprovado cerimonial protocolo if mg regulament...
Name: doc, dtype: object

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vetorização

tfidf_vectorizer = TfidfVectorizer(
    min_df=2 # Frequência mínima
)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['doc'])

tfidf_matrix.shape

(1000, 3663)

## Clusterização

Técnica de mineração de dados para agrupar dados automaticamente (aprendizado de máquina não-supervisionado), com base no seu grau de semelhança dos registros.

### Algoritmo K-Means

<img src="http://stanford.edu/~cpiech/cs221/img/kmeansViz.png">
**Fonte da figura:** http://stanford.edu/~cpiech/cs221/handouts/kmeans.html

In [39]:
from sklearn.cluster import KMeans

N_CLUSTERS = 25

kmeans = KMeans(
    n_clusters=N_CLUSTERS,
    max_iter=500,
    n_init=10,
    n_jobs=-1,
    random_state=1
)

kmeans.fit(tfidf_matrix)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=500,
    n_clusters=25, n_init=10, n_jobs=-1, precompute_distances='auto',
    random_state=1, tol=0.0001, verbose=0)

In [42]:
# Associa o id de cada cluster ao dataframe
df['clusters'] = kmeans.labels_

In [40]:
# Exibe o cluster dos 50 primeiros documentos
kmeans.labels_[:50]

array([20, 19, 10,  4, 20, 13, 12, 12, 20, 14, 20,  5, 15,  9,  4,  9,  0,
       10, 10, 10,  4, 14,  0,  2, 10,  9,  9,  0, 20,  0,  9,  3,  5, 16,
        3, 15, 16,  3, 15, 15,  3,  0, 10,  8, 20,  8, 15, 15, 15, 16],
      dtype=int32)

In [43]:
df[df['clusters'] == 0].head(5)['titulo']

16    Estudante do Campus Barbacena ganha medalha de...
22    Alunos do Campus Barbacena participam da cerim...
27    Obap em Barbacena: Apresentações artísticas e ...
29    Alunos do Curso Superior de Tecnologia em Sist...
41    Alunos do Campus Barbacena participam do IV En...
Name: titulo, dtype: object

In [52]:
# Exibe o título de 3 notícias de cada cluster
for i in range(N_CLUSTERS):
    print('Cluster', i + 1)
    print(df[df['clusters'] == i].head(3)['titulo'])
    print('\n----\n')

Cluster 1
16    Estudante do Campus Barbacena ganha medalha de...
22    Alunos do Campus Barbacena participam da cerim...
27    Obap em Barbacena: Apresentações artísticas e ...
Name: titulo, dtype: object

----

Cluster 2
158    Atenção formandos  dos Cursos de Nutrição e Li...
704    Divulgada a homologação do resultado da eleiçã...
705    Divulgada a homologação do resultado da eleiçã...
Name: titulo, dtype: object

----

Cluster 3
23     Cursos do Campus Barbacena recebem selo de qua...
106         Novo Processo Seletivo - EaD - IF Sudeste MG
121             Campus Barbacena reaproveita óleo usado 
Name: titulo, dtype: object

----

Cluster 4
31    Divulgada a seleção para Monitor da disciplina...
34    Divulgada a seleção de monitor para as discipl...
37    Divulgada a seleção de monitores paras discipl...
Name: titulo, dtype: object

----

Cluster 5
3     Divulgada a homologação das inscrições à candi...
14               Eleições CEPE: Homologação dos eleitos
20    Eleições CEPE:

## Comparação de algoritmos de clusterização

https://scikit-learn.org/stable/modules/clustering.html#overview-of-clustering-methods

<img src="https://scikit-learn.org/stable/_images/sphx_glr_plot_cluster_comparison_0011.png">

In [55]:
from sklearn.cluster import FeatureAgglomeration

fa = FeatureAgglomeration()
fa.fit_transform(tfidf_matrix.toarray())

array([[0.00064169, 0.        ],
       [0.00054253, 0.16098383],
       [0.0009236 , 0.        ],
       ...,
       [0.00084651, 0.1051223 ],
       [0.00096439, 0.08247019],
       [0.00141705, 0.        ]])

## Algoritmo Apriori

<img src="https://edumine.files.wordpress.com/2013/09/market-basket-analysis.png">

**Fonte da figura:** https://edumine.wordpress.com/2013/09/11/apriori-algorithm-simplified-with-an-example/

In [4]:
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder

dataset = [
    ['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
    ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
    ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
    ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
    ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']
]

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,Apple,Corn,Dill,Eggs,Ice cream,Kidney Beans,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,False,True,False,True,True,True,True,False,True
1,False,False,True,True,False,True,False,True,True,False,True
2,True,False,False,True,False,True,True,False,False,False,False
3,False,True,False,False,False,True,True,False,False,True,True
4,False,True,False,True,True,True,False,False,True,False,False


In [8]:
apriori(df, min_support=0.6)

,support,itemsets
0,0.8,(3)
1,1.0,(5)
2,0.6,(6)
3,0.6,(8)
4,0.6,(10)
5,0.8,"(3, 5)"
6,0.6,"(8, 3)"
7,0.6,"(5, 6)"
8,0.6,"(8, 5)"
9,0.6,"(10, 5)"


In [10]:
# Exibindo o nome dos itens (colunas)
apriori(df, min_support=0.6, use_colnames=True)

,support,itemsets
0,0.8,(Eggs)
1,1.0,(Kidney Beans)
2,0.6,(Milk)
3,0.6,(Onion)
4,0.6,(Yogurt)
5,0.8,"(Eggs, Kidney Beans)"
6,0.6,"(Eggs, Onion)"
7,0.6,"(Milk, Kidney Beans)"
8,0.6,"(Onion, Kidney Beans)"
9,0.6,"(Kidney Beans, Yogurt)"


## Atividade para casa

Implemente a técnica de modelagem de tópicos utilizando clusterização. 

O usuário deverá informar o número de assuntos, e o programa deverá retornar os 5 termos e os 5 documentos mais relevantes para cada assunto.

O desafio consiste no emprego de uma métrica para definir os documentos e os termos mais relevantes de cada cluster.